In [ ]:
# Import the req lib
import pandas as pd
import re
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer

import numpy as np
from keras.utils import np_utils
import sys

In [ ]:
path = "/content/sherlock-holmes.txt"
with open(path, "r") as file1:
    raw_text = file1.read()
    # print(raw_data)

In [ ]:
#Data Cleaning
raw_text = raw_text.lower()[:1000]
chars = sorted(list(set(raw_text)))
print(chars)

['\n', ' ', "'", '-', '1', '2', '3', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [ ]:
# reg = re.sub('[^a-zA-Z]',' ',demo)
bad_chars = [':',';','½']
for i in range(len(bad_chars)):
  text = raw_text.replace(bad_chars[i],"")
chars = sorted(list(set(text)))
print(chars)

['\n', ' ', "'", '-', '1', '2', '3', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [ ]:
textlen = len(text)
charlen = len(chars)


print("length is " + str(textlen))
print("length is " + str(charlen))

length is 1000
length is 29


In [ ]:
SEQ = 100
char_to_int = dict((c,i) for i,c in enumerate(chars))
data_x = []
data_y = []

for i in range(len(text) - SEQ):
  xtext = text[i:i+SEQ]
  X = [char_to_int[char] for char in xtext]
  data_x.append(X)
  Y = text[i+SEQ]
  data_y.append(char_to_int[Y])

In [ ]:
length = len(data_x)
data_x = np.array(data_x)
data_x = np.reshape(data_x,(data_x.shape[0],data_x.shape[1],1))
data_x = data_x/float(charlen)

data_y = np.array(data_y)
data_y = np_utils.to_categorical(data_y)
print(data_x.shape)
print(data_y.shape)

(900, 100, 1)
(900, 29)


In [ ]:
#Model building
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape = (SEQ, 1), return_sequences = True))
model.add(Dropout (0.2))
model.add(LSTM(256))
model.add(Dropout (0.2))
model.add(Dense(charlen,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
#Define the check point
filepath="text generation.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
history = model.fit(data_x, data_y, epochs=2, batch_size=128, callbacks = callbacks_list)

Epoch 1/2
8/8 [==============================] - ETA: 0s - loss: 2.1016
Epoch 1: loss improved from inf to 2.10156, saving model to text generation.h5
8/8 [==============================] - 15s 2s/step - loss: 2.1016
Epoch 2/2
8/8 [==============================] - ETA: 0s - loss: 2.0608
Epoch 2: loss improved from 2.10156 to 2.06084, saving model to text generation.h5
8/8 [==============================] - 15s 2s/step - loss: 2.0608


In [ ]:
#load the network weights
filename = 'text generation.h5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer ='adam')

In [ ]:
new_text ="Let us have some You perceive that the resulting has the appearance of pure water The proportion of blood cannot be more than one in a million As he spoke he threw into the vessel a few white crystals and then added some drops of "
new_text = new_text.lower()
chars = ['\n', ' ', "'", '-', '1', '2', '3', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
VOCABULARY=len(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
new_text = [char_to_int[c] for c in new_text]
int_to_char = dict((i, c) for i, c in enumerate(chars))
# test_text = [char_to_int[c] for c in new_text]
test_text = new_text

In [ ]:
generated_text = []
SEQ=100

if len(test_text) > 100: test_text =test_text[len(test_text)-100: ]
if len(test_text) < 100:
      pad = []
      space =  char_to_int[' ']
      pad = [space for i in range(100-len(test_text))]
      test_text = pad + test_text
for i in range(100):
      X = np.reshape(test_text, (1, SEQ, 1))
      X=X / float(VOCABULARY)
      Prediction = model.predict(X)
      index = np.argmax(Prediction)
      result = int_to_char[index]
      print("result is ",result)
      generated_text.append(result)
      test_text.append(index)
      test_text = test_text[1:]
output=' '.join(generated_text)

1/1 [==============================] - 0s 54ms/step
result is   
1/1 [==============================] - 0s 64ms/step
result is   
1/1 [==============================] - 0s 60ms/step
result is   
1/1 [==============================] - 0s 55ms/step
result is   
1/1 [==============================] - 0s 53ms/step
result is   
1/1 [==============================] - 0s 59ms/step
result is   
1/1 [==============================] - 0s 62ms/step
result is   
1/1 [==============================] - 0s 57ms/step
result is   
1/1 [==============================] - 0s 57ms/step
result is   
1/1 [==============================] - 0s 64ms/step
result is   
1/1 [==============================] - 0s 58ms/step
result is   
1/1 [==============================] - 0s 53ms/step
result is   
1/1 [==============================] - 0s 58ms/step
result is   
1/1 [==============================] - 0s 53ms/step
result is   
1/1 [==============================] - 0s 56ms/step
result is   
1/1 [====================

In [ ]:
print(output)